In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

### Train & Test Datasets

En el conjunto de datos de entrenamiento y de prueba aparece el tipo de mensaje junto con el mensaje. Para poder visualizar los datos correctamente, creamos un DataFrame a partir del *path* de ambos ficheros y añadimos los features correspondiente.

In [ ]:
columns = ['class', 'text']
train_dataset = pd.read_csv(train_file_path, names=columns ,delimiter='\t')
test_dataset = pd.read_csv(test_file_path, names=columns ,delimiter='\t')

train_dataset.head()

In [ ]:
test_dataset.head()

Procedemos a cambiar los valores categoricos de la clase por valores numericos. Segun las especificaciones: *ham = 0* y *spam = 1*.



In [ ]:
train_dataset['class'].replace(['ham','spam'], [0,1], inplace=True)
test_dataset['class'].replace(['ham','spam'], [0,1], inplace=True)

Separamos el label a predecir del resto de features

In [ ]:
train_labels = train_dataset.pop('class')
test_labels = test_dataset.pop('class')

### Codificación del texto & Preprocesamiento de los datos

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
VOCAB_SIZE = 8000

train_dataset = train_dataset['text'].values
test_dataset = test_dataset['text'].values

tokenizer = Tokenizer(num_words = VOCAB_SIZE)
tokenizer.fit_on_texts(train_dataset)

In [ ]:
coded_train_dataset = tokenizer.texts_to_sequences(train_dataset)
coded_test_dataset = tokenizer.texts_to_sequences(test_dataset)

VOCAB_SIZE = len(tokenizer.word_index) + 1
VOCAB_SIZE

In [ ]:
len(coded_train_dataset[0])

In [ ]:
len(coded_train_dataset[223])

Como se puede observar, las codificaciones de diferentes textos tienen longitudes distintas, lo cual es un problema ya que no podemos pasar datos de diferente longitud a nuestra red neuronal. Por esta razon, tenemos que tener todos los mensajes con la misma longitud.

In [ ]:
MAXLEN = 100

# Lo que conseguimos con esto es dar a todas las secuencias de texto la misma longitud
# Si el texto supera las 250 palabras, se eliminan las palabras extra
# Si el texto no llega a las 250 palabras, se insertan 0's al comienzo hasta llegar a las 250 palabras
data_train = keras.preprocessing.sequence.pad_sequences(coded_train_dataset, MAXLEN)
data_test = keras.preprocessing.sequence.pad_sequences(coded_test_dataset, MAXLEN)

### Creacion del modelo

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(VOCAB_SIZE, 32), # tam_vocabu, dimens_vector
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid') # Debido a que queremos valores entre 0 y 1
])

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['acc']
)

In [ ]:
model.summary()

### Entrenamiento del modelo

In [ ]:
history = model.fit(data_train, train_labels, epochs=10, validation_split=0.2)

### Evaluacion del modelo

In [ ]:
results = model.evaluate(data_test, test_labels)
results

Como se puede observar, obtenemos una precesion de 0.9878

### Predicciones

Obtenemos *word_index* en el que se encuentran las palabras con sus correspondientes codificaciones, teniendo la palabra mas comun el valor 1, la segunda mas comun el 2 y asi sucesivamente

In [ ]:
word_index = tokenizer.word_index
word_index

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  tokens = keras.preprocessing.text.text_to_word_sequence(pred_text) # convertimos el texto a tokens
  tokens = [word_index[word] if word in word_index else 0 for word in tokens] # lista con los tokens cifrados
  # Precesamos el texto
  coded_text = keras.preprocessing.sequence.pad_sequences([tokens], MAXLEN)[0]

  # Creamos un numpy de la dimension indicada para poder predecir de forma adecuada
  predictions = np.zeros((1,100))
  predictions[0] = coded_text

  result =  model.predict(predictions)
  prediction = result[0][0]

  # Segundo elemento de la lista en funcion del valor predicho -- ham(0) y spam(1)
  val = round(prediction)
  if val == 0: 
    clase = 'ham'
  else:
    clase = 'spam'

  return [prediction, clase]

pred_text = "wow, is your arm alright. that happened to me one time too"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
